**Importing all required libraries.**

In [92]:
import pandas as pd
import re
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.corpus import  wordnet
from nltk.corpus import stopwords
from nltk.stem import  WordNetLemmatizer
from nltk.corpus import  wordnet
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
import gensim.downloader as api
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


**Initializing lemmatizer.**

In [78]:
lemmatizer = WordNetLemmatizer()

**Loading the dataset.**

In [79]:
df = pd.read_csv('/content/dataset.csv')

**Data Preprocessing.**

In [87]:
class TextPreprocessor:
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.pos_map = {
            'J': wordnet.ADJ,
            'N': wordnet.NOUN,
            'V': wordnet.VERB,
            'R': wordnet.ADV
        }

    def get_wordnet_pos(self, word):
        tag = nltk.pos_tag([word])[0][1][0].upper()
        return self.pos_map.get(tag, wordnet.NOUN)


    def lower_text(self, text):
        return text.lower()

    #Removing mentions and URLs
    def remove_mentions(self, text):
        return re.sub(r'(@\w+|http\S+|www\S+|https\S+|#)', '', text)

    #Removing hashtags, special characters and numbers
    def remove_hashtags(self, text):
        return re.sub(r'#\W+', '', text)

    #Removing extra spaces
    def remove_extra_spaces(self, text):
        return re.sub(r'\s+', ' ', text).strip()

    def lemmatize_text(self, text):
        words = word_tokenize(text)
        return ' '.join(self.lemmatizer.lemmatize(w, self.get_wordnet_pos(w)) for w in words)

    def preprocess(self, text):
        cleaned_text = self.lower_text(text)
        cleaned_text = self.remove_mentions(cleaned_text)
        cleaned_text = self.remove_hashtags(cleaned_text)
        cleaned_text = self.remove_extra_spaces(cleaned_text)
        return self.lemmatize_text(cleaned_text)

#Preprocessing
preprocessor = TextPreprocessor()
df['cleaned_text'] = df['text'].apply(preprocessor.preprocess)

**Comparing original and cleaned text for few rows.**

In [88]:
for original, cleaned in zip(df['text'].head(), df['cleaned_text'].head()):
    print(f"Original: {original}")
    print(f"Cleaned: {cleaned}")
    print("-" * 50)

Original: When will the #NYSE #stockmarketcrash happen?
Cleaned: when will the nyse stockmarketcrash happen ?
--------------------------------------------------
Original: Aaj ka gyan:

If a company isn't a quality company, don't buy it just because the price is low. Bargain-bin companies often produce bargain-bin results.

#stockmarkets #stockmarketcrash #trading #investing #nifty #banknifty
Cleaned: aaj ka gyan : if a company be n't a quality company , do n't buy it just because the price be low . bargain-bin company often produce bargain-bin result . stockmarkets stockmarketcrash trading invest nifty banknifty
--------------------------------------------------
Original: The stock market needs to crash hard to make it realistic. You can't just keep printing money and keep blowing up a bubble its gonna pop one way or another. Smh sell when people are greedy and buy when people are fearful. #stockmarketcrash #economy #rich #Fed
Cleaned: the stock market need to crash hard to make it rea

**Splitting the dataset.**

In [93]:
# Features of cleaned text
X = df['cleaned_text']
# Labels
y = df['text_sentiment']

# Training: 80% and Testing: 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)

#Printing
print(f'Training set: {X_train.shape[0]} samples')
print(f'Testing set: {X_test.shape[0]} samples')

Training set: 27156 samples
Testing set: 6790 samples


**Initializing stopwords from NLTK.**

In [94]:
stop_words = stopwords.words('english')

**Bag of Words Technique for Sentiment Analysis.**

In [95]:
bow_vectorizer = CountVectorizer(max_features=5000, stop_words=stop_words)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)

#Selecting the top 100 features using the chi-square test.
chi2_selector = SelectKBest(chi2, k=100)

X_train_bow_chi2 = chi2_selector.fit_transform(X_train_bow, y_train)
X_test_bow_chi2 = chi2_selector.transform(X_test_bow)
feature_names_bow = bow_vectorizer.get_feature_names_out()

#Indices of the top 100 selected features from the chi-square selector.
top_100_indices = chi2_selector.get_support(indices=True)

#Top 100 features.
top_100_features_bow = [feature_names_bow[i] for i in top_100_indices]

# Printing the top 100 features
print("Top 100 features selected by chi-square test (BoW):")
for feature in top_100_features_bow:
    print(feature)

# SVM model using BoW with Chi-Square selected features.
model_svm_bow = SVC(kernel='linear')
model_svm_bow.fit(X_train_bow_chi2, y_train)

# Predicting and evaluating SVM model using BoW.
y_pred_svm_bow = model_svm_bow.predict(X_test_bow_chi2)
accuracy_svm_bow = accuracy_score(y_test, y_pred_svm_bow)
print(f'Accuracy using SVM with Bag of Words is: {accuracy_svm_bow:.4f}')

Top 100 features selected by chi-square test (BoW):
amp
asset
attack
back
bad
best
biden
blame
candle
close
collapse
crash
create
crisis
crude
day
de
drop
dump
easy
economy
enjoy
fall
fear
fearful
feel
friend
fuck
gain
get
give
go
good
great
greedy
growth
happy
hard
hell
help
high
hope
huge
important
inflation
interest
invest
investor
kill
le
like
lol
long
look
lose
loss
love
low
make
market
money
nifty
one
opportunity
others
pain
panic
people
pertains
play
please
positive
price
profit
project
ready
recession
rich
risk
russia
see
share
shit
stock
stop
strong
support
take
thank
time
top
ukraine
value
want
war
wealth
well
win
wrong
कर
Accuracy using SVM with Bag of Words is: 0.6988


**TF_IDF Technique for Sentiment Analysis.**

In [96]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words=stop_words)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

#Selecting the top 100 features using the chi-square test.
chi2_selector = SelectKBest(chi2, k=100)

# Applying the chi-square selection on TF-IDF features.
X_train_tfidf_chi2 = chi2_selector.fit_transform(X_train_tfidf, y_train)
X_test_tfidf_chi2 = chi2_selector.transform(X_test_tfidf)

# Getting feature names from the CountVectorizer.
feature_names_tfidf = tfidf_vectorizer.get_feature_names_out()

#Indices of the top 100 selected features from the chi-square selector.
top_100_indices = chi2_selector.get_support(indices=True)

#Top 100 features.
top_100_features_tfidf = [feature_names_tfidf[i] for i in top_100_indices]

# Printing the top 100 features.
print("Top 100 features selected by chi-square test (TF-IDF):")
for feature in top_100_features_tfidf:
    print(feature)

# SVM model using TF-IDF with Chi-Square selected features.
model_svm_tfidf = SVC(kernel='linear')
model_svm_tfidf.fit(X_train_tfidf_chi2, y_train)

# Predicting and evaluating SVM model (TF-IDF).
y_pred_svm_tfidf = model_svm_tfidf.predict(X_test_tfidf_chi2)
accuracy_svm_tfidf = accuracy_score(y_test, y_pred_svm_tfidf)
print(f'Accuracy using SVM with TF-IDF: {accuracy_svm_tfidf:.4f}')

Top 100 features selected by chi-square test (TF-IDF):
asset
attack
bad
bearmarket
best
blame
brutal
candle
collapse
crash
create
crisis
crude
damn
de
dead
destroy
drop
dump
easy
enjoy
fail
fake
fear
fearful
feel
free
friend
fuck
fun
gain
get
god
good
great
greedy
growth
happy
hard
hell
help
hope
huge
important
inflation
interest
kill
le
like
lol
look
lose
loss
love
low
negative
nice
nifty
opportunity
pain
panic
people
perfect
play
please
poor
positive
problem
profit
ready
recession
rich
risk
share
shit
smart
stockmarketcrash
stop
strong
suffer
support
sure
thank
thanks
top
ukraine
value
want
war
wealth
welcome
well
win
wish
wrong
wtf
yes
कर
नह
रह
Accuracy using SVM with TF-IDF: 0.7206


**Word2Vec Technique for Sentiment Analysis.**

In [97]:
# Loading pre-trained Word2Vec vectors from Google News pre-trained vectors 300 dimensions.
word2vec = api.load('word2vec-google-news-300')

# Function to convert a tweet to a Word2Vec vector by averaging word embeddings.
def word_to_vec(tweet, model, vector_size=300):
    words = tweet.split()
    word_vecs = [model[word] for word in words if word in model]
    if len(word_vecs) == 0:
        print(f"No word in the tweet is found in the model: {tweet}")
        return np.zeros(vector_size)
    return np.mean(word_vecs, axis=0)

# Applying the conversion to Word2Vec vectors for all tweets.
X = np.array([word_to_vec(tweet, word2vec) for tweet in df['cleaned_text']])
y = df['text_sentiment']

# Splitting the dataset into training: 80% and testing: 20% sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)

# Training a SVM model on the Word2Vec vectors.
model_w2v = SVC(kernel='linear')
model_w2v.fit(X_train, y_train)

# Predicting and evaluating the model.
y_pred_w2v = model_w2v.predict(X_test)
accuracy_w2v = accuracy_score(y_test, y_pred_w2v)
print(f'Accuracy using Word2Vec embeddings: {accuracy_w2v:.4f}')

No word in the tweet is found in the model: શેરબજાર માં મનુ મંદોડિયાનો આત્મા ઘૂસી ગયો હોય એવુ લાગે છે . stockmarketcrash
No word in the tweet is found in the model: stockmarketcrash
No word in the tweet is found in the model: stockmarketcrash
No word in the tweet is found in the model: stockmarketmemes stockmarketcrash stockmarketnews
No word in the tweet is found in the model: stockmarketcrash 🤔
No word in the tweet is found in the model: stockmarketcrash
No word in the tweet is found in the model: અડધુ બિસ્કિટ ચા માં ડૂબી જાય તો બીજા બિસ્કિટ થી બહાર કાઢવાની કોશિશ કરવી નહિ . બીજું પણ ડૂબી જ જશે ... કવિ અહીંયા ( શેર બજાર માં ) એવરેજ કરવાની ના પાડે છે .. stockmarketcrash
No word in the tweet is found in the model: stockmarketcrash
No word in the tweet is found in the model: stockmarketcrash
No word in the tweet is found in the model: yayyyyyyyyyyyyyyyyyyyyyeee stockmarketcrash
No word in the tweet is found in the model: dowjones stockmarketcrash stonk
No word in the tweet is found in th

**Glove for feature representation.**

In [98]:
!pip install torch==2.3.0 torchtext==0.18.0

In [99]:
import torch
import torchtext.vocab as vocab

# Loading pre-trained GloVe vectors from torchtext 300-dimensional vectors.
glove = vocab.GloVe(name='6B', dim=300)

# Function to convert a tweet to a GloVe vector by averaging word embeddings.
def tweet_to_glove_vec(tweet, model, vector_size=300):
    words = tweet.split()
    word_vecs = [model[word].numpy() for word in words if word in model.stoi]
    if len(word_vecs) == 0:
        print(f"No word in the tweet is found in the model: {tweet}")
        return np.zeros(vector_size)
    return np.mean(word_vecs, axis=0)

# Applying the conversion to GloVe vectors for all tweets.
X_glove = np.array([tweet_to_glove_vec(tweet, glove) for tweet in df['cleaned_text']])

# Splitting the dataset into training: 80% and testing: 20% sets.
X_train_glove, X_test_glove, y_train, y_test = train_test_split(X_glove, y, test_size=0.2, stratify=y, random_state=42)

# Training a Logistic Regression model on the GloVe vectors.
model_glove = SVC(kernel='linear')
model_glove.fit(X_train_glove, y_train)

# Predicting and evaluating the model.
y_pred_glove = model_glove.predict(X_test_glove)
accuracy_glove = accuracy_score(y_test, y_pred_glove)
print(f'Accuracy using GloVe embeddings: {accuracy_glove:.4f}')


No word in the tweet is found in the model: stockmarketcrash
No word in the tweet is found in the model: stockmarketcrash
No word in the tweet is found in the model: stockmarketmemes stockmarketcrash stockmarketnews
No word in the tweet is found in the model: stockmarketcrash 🤔
No word in the tweet is found in the model: stockmarketcrash
No word in the tweet is found in the model: stockmarketcrash
No word in the tweet is found in the model: stockmarketcrash
No word in the tweet is found in the model: yayyyyyyyyyyyyyyyyyyyyyeee stockmarketcrash
No word in the tweet is found in the model: stockmarketcrash
No word in the tweet is found in the model: welll stockmarketcrash
No word in the tweet is found in the model: stockmarketcrash
No word in the tweet is found in the model: stockmarketcrash nodickpussyhavingassbitch
No word in the tweet is found in the model: stockmarketcrash
No word in the tweet is found in the model: bidensamerica stockmarketcrash
No word in the tweet is found in the m

**Sentence Vectors Technique for Sentiment Analysis.**

In [103]:
import tensorflow as tf
import tensorflow_hub as hub

class TextPreprocessor:
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.pos_map = {
            'J': wordnet.ADJ,
            'N': wordnet.NOUN,
            'V': wordnet.VERB,
            'R': wordnet.ADV
        }
        # Loading Universal Sentence Encoder
        self.use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

    def get_wordnet_pos(self, word):
        tag = nltk.pos_tag([word])[0][1][0].upper()
        return self.pos_map.get(tag, wordnet.NOUN)

    def lower_text(self, text):
        return text.lower()

    #Removing mentions and URLs
    def remove_mentions(self, text):
        return re.sub(r'(@\w+|http\S+|www\S+|https\S+|#)', '', text)

    #Removing hashtags, special characters and numbers
    def remove_hashtags(self, text):
        return re.sub(r'#\W+', '', text)

    #Removing extra spaces
    def remove_extra_spaces(self, text):
        return re.sub(r'\s+', ' ', text).strip()

    def lemmatize_text(self, text):
        words = word_tokenize(text)
        return ' '.join(self.lemmatizer.lemmatize(w, self.get_wordnet_pos(w)) for w in words)

    def preprocess(self, text):
        cleaned_text = self.lower_text(text)
        cleaned_text = self.remove_mentions(cleaned_text)
        cleaned_text = self.remove_hashtags(cleaned_text)
        cleaned_text = self.remove_extra_spaces(cleaned_text)
        return self.lemmatize_text(cleaned_text)

    def get_sentence_vector(self, text):
        # Getting the USE embedding for the preprocessed text.
        embedding = self.use_model([text])
        # Converting to Python list.
        return embedding.numpy().tolist()[0]

preprocessor = TextPreprocessor()

# Applying preprocessing and getting sentence vectors.
df['cleaned_text'] = df['text'].apply(preprocessor.preprocess)
df['sentence_vector'] = df['cleaned_text'].apply(preprocessor.get_sentence_vector)

# Preparing the feature and target variables.
X = np.array(df['sentence_vector'].tolist())  # Convert sentence vectors to a numpy array
y = df['text_sentiment']  # Assuming sentiment labels are in 'text_sentiment'

# Splitting the data into training: 80% and test: 20% sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)

# Trainning an SVM classifier.
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

# Prediction on the test set.
y_pred = svm_model.predict(X_test)

# Calculating and printing the accuracy.
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy using SVM with Universal Sentence Encoder embeddings: {accuracy:.4f}")

Accuracy using SVM with Universal Sentence Encoder embeddings: 0.6663
